## 从定位参数到仅限关键字参数

### 调用函数时使用 * 和 **“展开”可迭代对象，映射到单个参数


In [56]:
def tag(name, *content, cls=None, **attrs):
    """生成一个或多个HTML标签"""
    if cls is not None:
        attrs['class'] = cls
    if attrs:
        attr_str = ''.join(' {0}={1}'.format(attr, value) 
                           for attr, value in sorted(attrs.items()))
    else:
        attr_str = ''
    if content:
        return "\t".join("<{0}{1}>{2}</{3}>".format(name, attr_str, c, name) for c in content)
    else:
        return '<{0}{1} />'.format(name, attr_str)

In [57]:
tag('br')

'<br />'

In [58]:
tag('p', 'hello', id=33)

'<p id=33>hello</p>'

In [59]:
tag('p', 'hello', 'world', cls='sidebar')

'<p class=sidebar>hello</p>\t<p class=sidebar>world</p>'

In [60]:
my_tag = {'name': 'img', 'title': 'Sunset Boulevard',
    'src': 'sunset.jpg', 'cls': 'framed'}
tag(**my_tag)

'<img class=framed src=sunset.jpg title=Sunset Boulevard />'